# 机器学习实验报告
## Lab6：Decision Tree Classifier
- 姓名：周钰宸
- 学号：2111408
- 专业：信息安全

## 1 实验要求
作业的提交格式参考之前的说明，提交到18329300691@163.com
### 1.1 基本要求
a)	基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；
b)	基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；
### 1.2 中级要求
a)  对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；
b)	对测试集Watermelon-test2进行预测，输出分类精度；
### 1.3高级要求
使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。

## 2 实验原理
### 2.1 决策树剪枝
剪枝是决策树算法中一个重要的概念，用于减少过拟合的风险并提高模型的泛化能力。剪枝有多种方法，最常见的两种是预剪枝（pre-pruning）和后剪枝（post-pruning）。预剪枝是在决策树构造过程中进行剪枝，而后剪枝是在决策树构造完成后进行剪枝。
### 2.1.1后剪枝（Post-Pruning）方法
后剪枝涉及到在决策树完全生成后，通过移除某些子树或节点来简化决策树。一个常用的后剪枝技术是基于验证集的性能来决定是否剪除某个节点（将其替换为最常见的类）。这种方法称为减少错误剪枝（Reduced Error Pruning）。

#### 2.1.1.1实现步骤：
1. **分割数据**：将数据分为训练集、测试集和验证集。
2. **构建完整的树**：使用训练集构建决策树，直到每个叶节点都纯净（或达到最小样本数）。
3. **剪枝**：对于树中的每个非叶节点，尝试将其替换为叶节点，并检查这种替换是否会在验证集上改善预测准确率。如果是，则进行剪枝。



## 3 基础要求
a) 基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；
b) 基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；

### 3.1 导入所需要的包

In [1]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
"""
@Project ：Machine Learning 
@File    ：main.py
@IDE     ：PyCharm 
@Author  ：ErwinZhou
@Date    ：2023/12/10 17:16 
"""

# ---初级要求---
import numpy as np
import pandas as pd
# ---中级要求---
import math
import copy
# ---高级要求---
# ---拓展要求---

### 3.2 导入数据集
基础要求中所需要的数据集是Watermelon-train1.csv和Watermelon-test1.csv。不过这里一口气统一的对所有数据集进行读取并展示，方便之后的使用。**首先打印在对离散值进行编码前的数据集样式。**

In [2]:
print("------------------Before encoding------------------")
df_train1 = pd.read_csv('data/Watermelon-train1.csv', encoding='gbk')
df_train2 = pd.read_csv('data/Watermelon-train2.csv', encoding='gbk')
df_test1 = pd.read_csv('data/Watermelon-test1.csv', encoding='gbk')
df_test2 = pd.read_csv('data/Watermelon-test2.csv', encoding='gbk')

print("------------------df_train1------------------\n", df_train1)
print(df_train1.shape)
print("------------------df_train2------------------\n", df_train2)
print(df_train2.shape)
print("------------------df_test1------------------\n", df_test1)
print(df_test1.shape)
print("------------------df_test2------------------\n", df_test2)
print(df_test2.shape)

------------------Before encoding------------------
------------------df_train1------------------
     编号  色泽  根蒂  敲声  纹理 好瓜
0    1  青绿  蜷缩  浊响  清晰  是
1    2  乌黑  蜷缩  沉闷  清晰  是
2    3  乌黑  蜷缩  浊响  清晰  是
3    4  青绿  蜷缩  沉闷  清晰  是
4    5  浅白  蜷缩  浊响  清晰  是
5    6  青绿  稍蜷  浊响  清晰  是
6    7  乌黑  稍蜷  浊响  稍糊  是
7    8  乌黑  稍蜷  浊响  清晰  是
8    9  乌黑  稍蜷  沉闷  稍糊  否
9   10  青绿  硬挺  清脆  清晰  否
10  11  浅白  硬挺  清脆  模糊  否
11  12  浅白  蜷缩  浊响  模糊  否
12  13  青绿  稍蜷  浊响  稍糊  否
13  14  浅白  稍蜷  沉闷  稍糊  否
14  15  浅白  蜷缩  浊响  模糊  否
15  16  青绿  蜷缩  沉闷  稍糊  否
(16, 6)
------------------df_train2------------------
     编号  色泽  根蒂  敲声  纹理     密度 好瓜
0    1  青绿  蜷缩  浊响  清晰  0.697  是
1    2  乌黑  蜷缩  沉闷  清晰  0.774  是
2    3  乌黑  蜷缩  浊响  清晰  0.634  是
3    4  青绿  蜷缩  沉闷  清晰  0.608  是
4    5  浅白  蜷缩  浊响  清晰  0.556  是
5    6  青绿  稍蜷  浊响  清晰  0.403  是
6    7  乌黑  稍蜷  浊响  稍糊  0.481  是
7    8  乌黑  稍蜷  浊响  清晰  0.437  是
8    9  乌黑  稍蜷  沉闷  稍糊  0.666  否
9   10  青绿  硬挺  清脆  清晰  0.243  否
10  11  浅白  硬挺  清脆  模糊  0.245  否
11  12  

可以看到以下特征：
* 本次实验数据集为中文，因此需要设置编码为gbk。
* train1和test1全部为离散值（正好适用于ID3）。而train2和test2的纹理列出现了连续值（正好适用于C4.5和CART）。同时为了后续方便处理，**对于就有离散值的train1和test1需要进行对应的编码。**
* 本次实验的数据集第一列为编号列，对后续没有作用，需要进行删除；
* 除此之外，**最重要的是：本次实验数据集数量级非常小，因此可以推测可能会对后续的决策树构造产生一定影响，理论上应该也不会出现过拟合的情况。**

### 3.3 数据预处理
基于上述分析，首先我们要做的就是对数据集进行预处理，主要包括删除编号列和**对数据集进行编码**两项任务。其中第二项任务完成后应该返回一个字典类型的变量，能够方便后续进行对照分析实验结果使用。

In [3]:
def encode_labels(df):
    """
    @description :Encode labels for discrete features
    @param df: dataframe
    @return df: the same df but with encoding labels
            mappings: dictionary of label and its encoding for looking up
    """
    mappings = {}
    for column in df.columns:
        if df[column].dtype == object:  # string for object
            df[column], unique_labels = pd.factorize(df[column])  # use factorize to encode
            mappings[column] = unique_labels
    return df, mappings


print("------------------After encoding------------------")
df_train1, mappings_df_train1 = encode_labels(df_train1.drop(['编号'], axis=1))
df_test1, mappings_df_test1 = encode_labels(df_test1.drop(['编号'], axis=1))

print("------------------df_train1------------------\n", df_train1)
print(mappings_df_train1)
print(df_train1.shape)
print("------------------df_test1------------------\n", df_test1)
print(mappings_df_test1)
print(df_test1.shape)


------------------After encoding------------------
------------------df_train1------------------
     色泽  根蒂  敲声  纹理  好瓜
0    0   0   0   0   0
1    1   0   1   0   0
2    1   0   0   0   0
3    0   0   1   0   0
4    2   0   0   0   0
5    0   1   0   0   0
6    1   1   0   1   0
7    1   1   0   0   0
8    1   1   1   1   1
9    0   2   2   0   1
10   2   2   2   2   1
11   2   0   0   2   1
12   0   1   0   1   1
13   2   1   1   1   1
14   2   0   0   2   1
15   0   0   1   1   1
{'色泽': Index(['青绿', '乌黑', '浅白'], dtype='object'), '根蒂': Index(['蜷缩', '稍蜷', '硬挺'], dtype='object'), '敲声': Index(['浊响', '沉闷', '清脆'], dtype='object'), '纹理': Index(['清晰', '稍糊', '模糊'], dtype='object'), '好瓜': Index(['是', '否'], dtype='object')}
(16, 5)
------------------df_test1------------------
    色泽  根蒂  敲声  纹理  好瓜
0   0   0   0   0   0
1   1   1   1   0   0
2   1   0   1   0   0
3   2   0   1   1   0
4   0   0   0   0   0
5   2   1   0   0   1
6   1   1   0   1   1
7   2   1   0   2   1
8   1   1   1   1   1

### 3.4 ID3决策树构造
于是来到了最关键的一步，现在我会定义一个DecisionTree类，具体实现如下：
1. 定义对应的方法，暂定方法只有一个method取值为ID3，未来会进一步拓展C4.5或者CART，**实现类似机制和策略的接口，方便使用；**
2. **calculate_entropy函数**：由于目前只有离散值的ID3，只实现对应的计算离散值的熵值。概率直接数数就行了。。。；
3. **calculate_information_gain**：调用calculate_entropy计算熵来计算基于某个特征的信息增益。返回的是feature对应特征的信息增益；
4. **best_feature_to_split**：根据此时中还剩下的特征列calculate_information_gain的返回结果，并返回其中**最大值对应的特征，即所需要选择的特征；**
5. **id3：实现对应的ID3算法，具体如下。**
* 计算当前节点的父节点类别，这是通过找出当前数据集中最常见的类别来实现的
* 用best_feature_to_split函数来找出信息增益最大的特征，这将是我们用来分割数据集的特征
* 创建了一个新的字典来表示决策树，字典的键是最佳特征的名字，值是一个空字典，这个空字典稍后将被填充为子树
* 从特征列表中移除了最佳特征，因为我们已经使用了这个特征进行分割，所以在子树中不再需要它
* 最后对于最佳特征的每个唯一值，创建了一个子数据集，这个子数据集只包含最佳特征等于该值的样本。然后递归地调用id3函数在这个子数据集上构建子树，并将子树添加到决策树中
* **递归截止条件一共有三条：**
（1）当前结点所有样本都属于同⼀类别；
（2）当前结点的所有属性值都相同，没有剩余属性可用来进一步划分样本；
（3）达到叶子结点的最小样本数；
**其中考虑到由于数据特征较小，因此树不会非常深，故没有设计达到最大树深这一停止条件。**
6. **predict**：模型预测方法，对每一行应用调用predict_single。即对每一个实例预测。
7. **predict_single**：遍历语法树找到对应的分类结果，只要达到叶节点即可。
8. **calculate_accuracy**:计算准确率。

In [4]:
class DecisionTree:
    """
    @description :Decision Tree Classifier
    @param method: For future extension to other methods, default for ID3
    """

    def __init__(self, method='ID3'):
        """
        @description :Initialize the classifier
        @param method: prediction method
        """
        self.method = method
        self.tree = None

    def calculate_entropy(self, y):
        """
        @description: calculate entropy for
        @param y:
        @return:
        """
        # discrete values, count the bin is enough
        class_counts = np.bincount(y)
        probabilities = class_counts / len(y)
        entropy = -np.sum([p * np.log2(p) for p in probabilities if p > 0])
        return entropy

    def calculate_information_gain(self, X, y, feature):
        """
        @description: calculate the information and store in a dictionary for future looking up
        @param X:
        @param y:
        @param feature:
        @return:
        """
        total_entropy = self.calculate_entropy(y)
        values, counts = np.unique(X[feature], return_counts=True)
        weighted_entropy = np.sum(
            [(counts[i] / np.sum(counts)) * self.calculate_entropy(y[X[feature] == v]) for i, v in enumerate(values)])
        # Calculate the difference to measure the information gain
        information_gain = total_entropy - weighted_entropy
        return information_gain

    def best_feature_to_split(self, X, y):
        """
        @description :Find the best feature to split which has the biggest information gain
        @param X:
        @param y:
        @return: the name of the feature has the biggest information gain
        """
        information_gains = {
            feature: self.calculate_information_gain(X, y, feature)
            for feature in X.columns}
        # max on biggest value of information gain(not the key) in the dictionary
        return max(information_gains, key=information_gains.get)

    def id3(self, X, y, original_data, features, parent_node_class=None):
        """
        @description :ID3 algorithm
        @param X:
        @param y:
        @param original_data:
        @param features:
        @param parent_node_class:
        @return:
        """
        # Firstly, it can stop at three conditions(Ignore the situation that the tree reaches a certain depth):
        if len(np.unique(y)) <= 1:
            # 1. If there is only one class left, return it
            return np.unique(y)[0]
        elif len(y) == 0:
            # 2. If there is no data left, return the most common class
            return np.unique(original_data["好瓜"])[
                np.argmax(np.unique(original_data["好瓜"],
                                    return_counts=True)[1])]
        elif len(features) == 0:
            # 3. If there is no feature left, return the parent node class（which is the most common class）
            return parent_node_class
        else:
            # Then, If none of the above conditions is satisfied, continue to split
            parent_node_class = np.unique(y)[
                np.argmax(np.unique(y, return_counts=True)[1])]
            best_feature = self.best_feature_to_split(X, y)
            tree = {best_feature: {}}
            features = [i for i in features if i != best_feature]
            for value in np.unique(X[best_feature]):
                sub_data = X.where(X[best_feature] == value).dropna()
                subtree = self.id3(sub_data, y[sub_data.index], original_data, features, parent_node_class)
                tree[best_feature][value] = subtree
            return tree

    def fit(self, X, y):
        """
        @description: train the model based on different given methods
        @param X:
        @param y:
        @return:
        """
        features = X.columns.tolist()
        if self.method == 'ID3':
            self.tree = self.id3(X, y, pd.concat([X, y], axis=1), features)
        else:
            # to be continue......
            raise Exception('Unknown method.')

    def predict(self, X):
        """
        @description: using the re-trained model to predict the result
        @param X:
        @return:
        """
        # apply predict_single on every row of X
        predictions = X.apply(self.predict_single, axis=1)
        return predictions

    def predict_single(self, x):
        """
        @description: This method is used to predict the class label for a single instance 'x'.
        @param x:
        @return:
        """
        # Traverses the decision tree on the values in 'x' until it reaches a leaf node.
        tree = self.tree
        while isinstance(tree, dict):
            root = next(iter(tree))
            tree = tree[root][x[root]]
        # The class label of the leaf node is returned as the prediction for 'x'.
        return tree

    def calculate_accuracy(self, y_true, y_pred):
        """
        @description: calculate accuracy for true labels and predicted ones
        @param y_true:
        @param y_pred:
        @return:
        """
        return np.sum(y_true == y_pred) / len(y_true)

### 3.5 训练决策树构造，预测并输出分类精度
在此处同时实现通过模型的训练构建决策树，并输出对应的决策树样貌。同时，通过模型的预测，输出对应的分类精度。

In [5]:
# split the data
X_train = df_train1.drop(['好瓜'], axis=1)
y_train = df_train1['好瓜']
X_test = df_test1.drop(['好瓜'], axis=1)
y_test = df_test1['好瓜']

# Create Decision Tree Model and train it
Decision_Tree_Classifier_ID3 = DecisionTree("ID3")
Decision_Tree_Classifier_ID3.fit(X_train, y_train)

# Predict and output the accuracy
y_pred = Decision_Tree_Classifier_ID3.predict(X_test)
accuracy = Decision_Tree_Classifier_ID3.calculate_accuracy(y_test, y_pred)
# split the data
X_train = df_train1.drop(['好瓜'], axis=1)
y_train = df_train1['好瓜']
X_test = df_test1.drop(['好瓜'], axis=1)
y_test = df_test1['好瓜']

# Create Decision Tree Model and train it
Decision_Tree_Classifier_ID3 = DecisionTree("ID3")
Decision_Tree_Classifier_ID3.fit(X_train, y_train)

# Predict and output the accuracy and error rate
y_pred = Decision_Tree_Classifier_ID3.predict(X_test)
accuracy = Decision_Tree_Classifier_ID3.calculate_accuracy(y_test, y_pred) 
error_rate = (1 - accuracy) * 100
accuracy = accuracy * 100
print("------------------ID3 Decision Tree------------------")
print(Decision_Tree_Classifier_ID3.tree)
print("Classify accuracy for Decision Tree Classifier ID3 is: {:.2f}%".format(accuracy))
print("Classify error rate for Decision Tree Classifier ID3 is: {:.2f}%".format(error_rate))


------------------ID3 Decision Tree------------------
{'纹理': {0: {'根蒂': {0.0: 0, 1.0: 0, 2.0: 1}}, 1: {'色泽': {0.0: 1, 1.0: {'敲声': {0.0: 0, 1.0: 1}}, 2.0: 1}}, 2: 1}}
Classify accuracy for Decision Tree Classifier ID3 is: 70.00%
Classify error rate for Decision Tree Classifier ID3 is: 30.00%


### 3.6 结果分析
初级要求中，我们构造了对应的ID3决策树并且基于Watermelon-test1进行了预测。虽然出于时间原因，没有来得及对决策树进行一些可视化呈现，不过可以看到输出的决策树大概摸样。**结果发现分类精度还可以，达到了70%，错误率仅有30%。证明分类器实现较为成功。**

## 4 中级要求
a)  对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；
b)	对测试集Watermelon-test2进行预测，输出分类精度；

### 4.1 C4.5改进
由于C4.5和之前构造的ID3决策树的区别主要在于C4.5可以处理连续型属性，因此我们需要对之前的代码进行一定的改进。**主要改进的地方有以下几个：**
1. **连续特征的处理**：
   - ID3算法仅能处理离散特征，而C4.5扩展了这一点，能够处理连续特征。
   - `best_feature_to_split_c4_5`函数针对连续特征寻找最佳分割点，并计算基于这些分割点的信息增益。(这里由于其实现和ID3算法的`best_feature_to_split_id3`函数参数等都不相同，因此我暂时没有把二者合二为一。)

2. **信息增益比**：
   - C4.5使用信息增益比（Information Gain Ratio）而不是仅仅使用信息增益（Information Gain），这有助于减少对具有较多值的特征的偏好。
   - `calculate_information_gain` 方法中，信息增益的计算被用于ID3，而对于C4.5，我在通过调用修改的`calculate_entropy`来计算信息增益比。

3. **不同的树构建逻辑**：
   - 在`c4_5`方法中，根据特征是否连续使用不同的逻辑来构建树。对于连续特征，根据计算出的最佳分割点将数据分成两部分，并对每部分递归地构建子树。

4. **预测逻辑的改变**：
   - 在`predict_single`方法中，我添加了处理连续特征的逻辑。这包括检查特征值是否小于或大于分割点，并相应地遍历决策树。

5. **数据过滤方法**：
   - 实现了`filter_data`和`filter_data_by_threshold`方法来处理离散和连续特征的数据过滤。这些方法在树的构建过程中用于根据特征值分割数据集。
6. **其它**：
   - 除此之外，还有一些小的改进，比如我在DecisionTree类多添加了`self.features`和`self.features_properties`，**用于拓展针对于不同离散或者连续类型值的处理。**

In [6]:
class DecisionTree:
    """
    @description :Decision Tree Classifier
    @param method: For future extension to other methods, default for ID3
    """

    def __init__(self, method, features, features_properties):
        """
        @description :Initialize the classifier
        @param method: prediction method
        @param features: AKA titles for each column
        @param features_properties: feature properties(0 for discrete, 1 for continuous)
        """
        self.features_properties = features_properties
        self.features = features
        self.method = method
        self.tree = None

    def most_common(self, data):
        """
        @description: calculate the most common label in the last column
        @param data:
        @return:
        """
        # Extract the labels from the last column of the data
        labels = data.iloc[:, -1]
        # Sort the labels by their frequency in descending order
        sorted_labels = labels.value_counts(sort=True)
        # Return the most common label
        return sorted_labels.keys()[0]

    def filter_data(self, data, target_value, target_index):
        """
        @description: filter the data based on a target value and index
        @param data:
        @param target_value:
        @param target_index:
        @return: a new data with entries that match the target value, excluding the target itself from each entry.
        """
        filtered_data = []
        for data_entry in data:
            if data_entry[target_index] == target_value:
                filtered_entry = data_entry[:target_index] + data_entry[target_index + 1:]
                filtered_data.append(filtered_entry)
        return filtered_data

    def filter_data_by_threshold(self, data, feature_index, threshold, option='S'):
        """
        @description: filter the data based on a threshold and index
                      If option is 'S', it returns entries with feature value less than the threshold.
                      If option is not 'S', it returns entries with feature value greater than the threshold.
        @param data:
        @param feature_index:
        @param threshold:
        @param option:
        @return:
        """
        # Initialize an empty list to store the filtered data
        filtered_data = []
        for data_entry in data:
            # If the option is 'S', check if the feature value is less than the threshold
            if option == 'S':
                if float(data_entry[feature_index]) < threshold:
                    # If the feature value is less than the threshold, add the entry to the filtered data
                    filtered_data.append(data_entry)
            else:
                # If the option is not 'S', check if the feature value is greater than the threshold
                if float(data_entry[feature_index]) > threshold:
                    # If the feature value is greater than the threshold, add the entry to the filtered data
                    filtered_data.append(data_entry)

        return filtered_data

    def calculate_entropy(self, data):
        """
        @description: calculate entropy for
        @param data:
        @return:
        """
        if self.method == 'ID3':
            # discrete values, count the bin is enough
            class_counts = np.bincount(data)
            probabilities = class_counts / len(data)
            entropy = -np.sum([p * np.log2(p) for p in probabilities if p > 0])
            return entropy
        elif self.method == 'C4.5':
            # Calculate the total number of entries in the data
            total_entries = len(data)
            # Initialize a dictionary to store the frequency of each label
            label_frequencies = {}
            # Iterate over each entry in the data
            for data_entry in data:
                # Get the label of the entry
                data_label = data_entry[-1]
                # If the label is not in the dictionary, initialize it with 0
                if data_label not in label_frequencies:
                    label_frequencies[data_label] = 0
                # Increment the count of the label in the dictionary
                label_frequencies[data_label] += 1
            # Initialize entropy with 0
            calculated_entropy = 0
            # Calculate the entropy
            for label_count in label_frequencies.values():
                # Calculate the probability of the label
                probability = label_count / total_entries
                # Update the entropy
                calculated_entropy -= probability * math.log2(probability)
            # Return the calculated entropy
            return calculated_entropy

        else:
            raise Exception("No such method!")

    def calculate_information_gain(self, X, y, feature):
        """
        @description: calculate the information and store in a dictionary for future looking up
        @param X:
        @param y:
        @param feature:
        @return:
        """
        total_entropy = self.calculate_entropy(y)
        values, counts = np.unique(X[feature], return_counts=True)
        weighted_entropy = np.sum(
            [(counts[i] / np.sum(counts)) * self.calculate_entropy(y[X[feature] == v]) for i, v in enumerate(values)])
        # Calculate the difference to measure the information gain
        information_gain = total_entropy - weighted_entropy
        return information_gain

    def best_feature_to_split_id3(self, X, y):
        """
        @description :Find the best feature to split which has the biggest information gain
        @param X:
        @param y:
        @return: the name of the feature has the biggest information gain
        """
        information_gains = {
            feature: self.calculate_information_gain(X, y, feature)
            for feature in X.columns}
        # max on biggest value of information gain(not the key) in the dictionary
        return max(information_gains, key=information_gains.get)

    def best_feature_to_split_c4_5(self, X, fes_prpers):
        """
        This function chooses the best feature for splitting the dataset based on information gain.
        It correctly handles both discrete and continuous features.
        """
        num_of_features = len(fes_prpers)
        base_entropy_value = self.calculate_entropy(X)
        max_info_gain = 0.0
        best_feature_index = -1
        best_partition_val = None
        for i in range(num_of_features):
            feature_values = [example[i] for example in X]
            unique_feature_values = set(feature_values)
            entropy_value = 0.0
            best_partition_val_i = None
            if fes_prpers[i] == 0:  # Discrete feature
                for value in unique_feature_values:
                    filtered_data = self.filter_data(X, value, i)
                    probability = len(filtered_data) / float(len(X))
                    entropy_value += probability * self.calculate_entropy(filtered_data)
            else:  # Continuous feature
                sorted_unique_feature_values = list(unique_feature_values)
                sorted_unique_feature_values.sort()
                min_entropy_value = 10000
                for j in range(len(sorted_unique_feature_values) - 1):
                    partition_val = (float(sorted_unique_feature_values[j]) + float(
                        sorted_unique_feature_values[j + 1])) / 2
                    left_data = self.filter_data_by_threshold(X, i, partition_val, 'S')
                    right_data = self.filter_data_by_threshold(X, i, partition_val, 'T')
                    left_probability = len(left_data) / float(len(X))
                    right_probability = len(right_data) / float(len(X))
                    entropy_val = left_probability * self.calculate_entropy(left_data) + right_probability * self.calculate_entropy(
                        right_data)
                    if entropy_val < min_entropy_value:
                        min_entropy_value = entropy_val
                        best_partition_val_i = partition_val
                entropy_value = min_entropy_value
            information_gain = base_entropy_value - entropy_value
            if information_gain > max_info_gain:
                max_info_gain = information_gain
                best_feature_index = i
                best_partition_val = best_partition_val_i
        return best_feature_index, best_partition_val




    def id3(self, X, y, original_data, features, parent_node_class=None):
        """
        @description :ID3 algorithm
        @param X:
        @param y:
        @param original_data:
        @param features:
        @param parent_node_class:
        @return:
        """
        # Firstly, it can stop at three conditions(Ignore the situation that the tree reaches a certain depth):
        if len(np.unique(y)) <= 1:
            # 1. If there is only one class left, return it
            return np.unique(y)[0]
        elif len(y) == 0:
            # 2. If there is no data left, return the most common class
            return np.unique(original_data["好瓜"])[
                np.argmax(np.unique(original_data["好瓜"],
                                    return_counts=True)[1])]
        elif len(features) == 0:
            # 3. If there is no feature left, return the parent node class（which is the most common class）
            return parent_node_class
        else:
            # Then, If none of the above conditions is satisfied, continue to split
            parent_node_class = np.unique(y)[
                np.argmax(np.unique(y, return_counts=True)[1])]
            best_feature = self.best_feature_to_split_id3(X, y)
            tree = {best_feature: {}}
            features = [i for i in features if i != best_feature]
            for value in np.unique(X[best_feature]):
                sub_data = X.where(X[best_feature] == value).dropna()
                subtree = self.id3(sub_data, y[sub_data.index], original_data, features, parent_node_class)
                tree[best_feature][value] = subtree
            return tree

    def c4_5(self, X, fes, fe_propers):
        """
        @description: C4.5 algorithm
        @param X:
        @param fes:
        @param fe_propers:
        @return:
        """
        # Get the class labels for each data entry
        classList = [example[-1] for example in X]
        # If all class labels are the same, return the class label
        if classList.count(classList[0]) == len(classList):
            return classList[0]
        # If there is only one feature left, return the most common class label
        if len(X[0]) == 1:
            return self.most_common(classList)

        # Choose the best feature and partition value for splitting
        bestFeature, bestPartitionValue = self.best_feature_to_split_c4_5(X, fe_propers)

        # If no best feature is found, return the most common class label
        if bestFeature == -1:
            return self.most_common(classList)

        # If the best feature is discrete
        if fe_propers[bestFeature] == 0:
            bestFeatureLabel = fes[bestFeature]
            Tree = {bestFeatureLabel: {}}
            newLabels = copy.copy(fes)
            newLabelProperties = copy.copy(fe_propers)
            del (newLabels[bestFeature])
            del (newLabelProperties[bestFeature])
            featureValues = [example[bestFeature] for example in X]
            uniqueValues = set(featureValues)
            for value in uniqueValues:
                subLabels = newLabels[:]
                subLabelProperties = newLabelProperties[:]
                Tree[bestFeatureLabel][value] = self.c4_5(self.filter_data(X, value, bestFeature), subLabels,
                                                          subLabelProperties)
        else:  # If the best feature is continuous
            bestFeatureLabel = fes[bestFeature] + '<' + str(bestPartitionValue)
            Tree = {bestFeatureLabel: {}}
            subLabels = fes[:]
            subLabelProperties = fe_propers[:]
            # Build the left subtree
            leftValue = '是'
            Tree[bestFeatureLabel][leftValue] = self.c4_5(
                self.filter_data_by_threshold(X, bestFeature, bestPartitionValue, 'S'), subLabels,
                subLabelProperties)
            # Build the right subtree
            rightValue = '否'
            Tree[bestFeatureLabel][rightValue] = self.c4_5(
                self.filter_data_by_threshold(X, bestFeature, bestPartitionValue, 'T'), subLabels,
                subLabelProperties)
        return Tree

    def fit(self, X, y):
        """
        @description: train the model based on different given methods
        @param X:
        @param y:
        @return:
        """

        if self.method == 'ID3':
            features = X.columns.tolist()
            self.tree = self.id3(X, y, pd.concat([X, y], axis=1), features)
        elif self.method == 'C4.5':
            self.tree = self.c4_5(X, self.features, self.features_properties)
        else:
            raise Exception("No such method!")

    def predict(self, X):
        """
        @description: using the re-trained model to predict the result
        @param X:
        @return:
        """
        if self.method == 'ID3':
            # apply predict_single on every row of X
            predictions = X.apply(self.predict_single, axis=1)
            return predictions
        elif self.method == 'C4.5':
            predictions = []
            for x in X:
                predictions.append(self.predict_single(self.tree, self.features, self.features_properties, x))
        else:
            raise Exception("No such method!")

        return predictions
    def predict_single(self, DecisionTree, fes, fe_propers, test_data):
        """
        @description: using the re-trained model to predict the result
        @param DecisionTree: the whole decision tree or subtree
        @param fes: feature labels
        @param fe_propers: feature properties
        @param test_data: the test data
        @return:
        """
        if self.method == 'ID3':
            # Traverses the decision tree on the values in 'x' until it reaches a leaf node.
            tree = DecisionTree
            while isinstance(tree, dict):
                root = next(iter(tree))
                tree = tree[root][test_data[root]]
            # The class label of the leaf node is returned as the prediction for 'x'.
            return tree
        elif self.method == 'C4.5':
            # Get the first key of the input tree
            firstKey = list(DecisionTree.keys())[0]
            featureName = firstKey
            # Check if the feature is continuous
            splitIndex = str(firstKey).find('<')
            if splitIndex > -1:
                featureName = str(firstKey)[:splitIndex]
            # Get the subtree corresponding to the first key
            subTree = DecisionTree[firstKey]
            # Get the index of the feature in the feature labels list
            featureIndex = fes.index(featureName)
            predictedLabel = None
            for key in subTree.keys():
                if fe_propers[featureIndex] == 0:
                    if test_data[featureIndex] == key:
                        if type(subTree[key]).__name__ == 'dict':
                            predictedLabel = self.predict_single(subTree[key], fes, fe_propers, test_data)
                        else:
                            predictedLabel = subTree[key]
                else:
                    partitionValue = float(str(firstKey)[splitIndex + 1:])
                    if test_data[featureIndex] < partitionValue:
                        if type(subTree['是']).__name__ == 'dict':
                            predictedLabel = self.predict_single(subTree['是'], fes, fe_propers, test_data)
                        else:
                            predictedLabel = subTree['是']
                    else:
                        if type(subTree['否']).__name__ == 'dict':
                            predictedLabel = self.predict_single(subTree['否'], fes, fe_propers, test_data)
                        else:
                            predictedLabel = subTree['否']

            return predictedLabel
        else:
            raise Exception("No such method!")

    def calculate_accuracy(self, y_true, y_pred):
        """
        @description: calculate accuracy for true labels and predicted ones
        @param y_true:
        @param y_pred:
        @return:
        """
        return np.sum(y_true == y_pred) / len(y_true)

### 4.2 数据预处理与决策树构建
在重新改进了我们的决策树`DecisionTree`后，它如今**可以根据不同的模型给定方法适用于不同的类型数据集了。**不过在构建决策树之前，这次我需要传入记录特征类型的`features_properties`，**用于拓展针对于不同离散或者连续类型值的处理。** 

同时也要对数据进行一些处理，而不是像之前的可以简单的进行编码即可。**在处理完后，只需要传进参数`C4.5`，`features`和`features_properties`然后训练即可。**

最后构建完成的决策树保留在`Decision_Tree_Classifier_C4_5.tree`中。输出出来结果如下。

In [7]:
train2 = np.array(df_train2).tolist()
new_train2 = []
for i in train2:
    new_train2.append(i[1:])
train2 = new_train2

features = ['色泽', '根蒂', '敲声', '纹理', '密度']
# 0 for discrete, 1 for continuous
feature_properties = [0, 0, 0, 0, 1]

# Create Decision Tree Model and train it
Decision_Tree_Classifier_C4_5 = DecisionTree("C4.5", features, feature_properties)

Decision_Tree_Classifier_C4_5.fit(train2, df_train2['好瓜'])

print("------------------C4.5 Decision Tree------------------")
print(Decision_Tree_Classifier_C4_5.tree)

------------------C4.5 Decision Tree------------------
{'纹理': {'模糊': '否', '清晰': {'密度<0.3815': {'是': '否', '否': '是'}}, '稍糊': {'密度<0.56': {'是': '是', '否': '否'}}}}


### 4.3 预测并输出分类精度
现在我们只需对test2进行相同的数据处理后，调用`Decision_Tree_Classifier_C4_5.predict`方法即可。**最后输出了分类的精度结果如下：**

In [8]:
test2 = np.array(df_test2).tolist()
new_test2 = []
for i in test2:
    new_test2.append(i[1:])
test2 = new_test2


# Predict and output the accuracy
y_pred = Decision_Tree_Classifier_C4_5.predict(test2)
accuracy = Decision_Tree_Classifier_C4_5.calculate_accuracy(df_test2['好瓜'], y_pred)
error_rate = (1 - accuracy) * 100
accuracy = accuracy * 100
print("Classify accuracy for Decision Tree Classifier C4.5 is: {:.2f}%".format(accuracy))
print("Classify error rate for Decision Tree Classifier C4.5 is: {:.2f}%".format(error_rate))

Classify accuracy for Decision Tree Classifier C4.5 is: 80.00%
Classify error rate for Decision Tree Classifier C4.5 is: 20.00%


### 4.4 结果分析
中级要求中，我们构造了对应的C4.5决策树并且基于Watermelon-test2进行了预测。**结果发现分类精度也还可以，达到了80%，错误率仅有20%。证明C4.5分类器实现较为成功。**

同时其分类效果也优于ID3决策树，这也是由于C4.5能够处理连续型属性的优势所在。~~(虽然它们用的根本不是一个数据集吧）~~

## 5 高级要求
使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。

### 5.1 后剪枝方法实现
首先我们之前已经介绍过了后剪枝方法，现在尝试先实现基本的DecisionTree的后剪枝方法，然后再将其封装进我的DecisionTree类中。**实现的具体思路如下：主要通过验证数据集来剪枝以提高模型的泛化性能。**

1. **递归遍历树：** 代码首先检查当前节点是否是叶子节点（不是一个字典类型的节点）。如果是叶子节点，表示已经到达决策树的底部，不需要再剪枝，直接返回该叶子节点。

2. **递归遍历子节点：** 如果当前节点不是叶子节点，代码会进一步递归遍历当前节点的子节点（字典中的键值对）。这是一个深度优先搜索的过程，将对每个子节点执行相同的操作。

3. **计算原始精度和剪枝精度：** 对于每个子节点，代码会计算在验证数据集上的原始精度（original_accuracy），这是在未剪枝时的精度。然后，它使用之前的 `most_common` 计算该子节点的最常见的类别值，并用这个值来替代子节点的决策，从而进行剪枝。接着，它再次计算在验证数据集上的剪枝后的精度（pruned_accuracy）。

4. **判断是否剪枝：** 最后，代码会比较剪枝后的精度（pruned_accuracy）与原始精度（original_accuracy）。如果剪枝后的精度大于或等于原始精度，说明剪枝后的决策树更具泛化性能，可以进行剪枝操作，将当前节点替换为叶子节点，并返回剪枝后的节点。否则，保持原样返回当前节点，不进行剪枝。

这个后剪枝过程通过递归地检查每个子节点，并根据验证数据集的性能来决定是否剪枝，有助于减小决策树的复杂度，提高模型的泛化性能。**主要用于防止过拟合。**

In [9]:
def post_prune(self, node, validation_data):
    """
    @description: Post-pruning method for the decision tree
    @param node: current node
    @param validation_data: validation data set
    @return: pruned decision tree
    """
    # Check if the current node is a leaf node
    if not isinstance(node, dict):
        return node

    # Recursively prune child nodes
    for key in node.keys():
        node[key] = self.post_prune(node[key], validation_data)

    # Check if the current node can be pruned
    original_accuracy = self.calculate_accuracy(validation_data['label'], self.predict(validation_data))
    pruned_node = self.most_common(validation_data['label'])
    pruned_accuracy = self.calculate_accuracy(validation_data['label'], self.predict(validation_data, pruned_node))

    # If the accuracy improves after pruning, prune the node
    if pruned_accuracy >= original_accuracy:
        return pruned_node
    else:
        return node

### 5.2 决策树后剪枝封装
然后我们只需要将其加入到`fit方法`中，再ID3和C4.5正常的决策树训练构建结束后，直接开始后剪枝构造真正树结果。最后再稍微修改下predict和predict_single的一些细节，目的就是实现对部分子树的预测。**封装后的完整DecisionTree类如下：**


In [10]:
class DecisionTree:
    """
    @description :Decision Tree Classifier
    @param method: For future extension to other methods, default for ID3
    """

    def __init__(self, method, features, features_properties, puring=False):
        """
        @description :Initialize the classifier
        @param method: prediction method
        @param features: AKA titles for each column
        @param features_properties: feature properties(0 for discrete, 1 for continuous)
        @param puring: whether to use puring and which method to implement puring
        """
        self.puring = puring
        self.features_properties = features_properties
        self.features = features
        self.method = method
        self.tree = None

    def most_common(self, data):
        """
        @description: calculate the most common label in the last column
        @param data:
        @return:
        """
        # Extract the labels from the last column of the data
        labels = data.iloc[:, -1]
        # Sort the labels by their frequency in descending order
        sorted_labels = labels.value_counts(sort=True)
        # Return the most common label
        return sorted_labels.keys()[0]

    def filter_data(self, data, target_value, target_index):
        """
        @description: filter the data based on a target value and index
        @param data:
        @param target_value:
        @param target_index:
        @return: a new data with entries that match the target value, excluding the target itself from each entry.
        """
        filtered_data = []
        for data_entry in data:
            if data_entry[target_index] == target_value:
                filtered_entry = data_entry[:target_index] + data_entry[target_index + 1:]
                filtered_data.append(filtered_entry)
        return filtered_data

    def filter_data_by_threshold(self, data, feature_index, threshold, option='S'):
        """
        @description: filter the data based on a threshold and index
                      If option is 'S', it returns entries with feature value less than the threshold.
                      If option is not 'S', it returns entries with feature value greater than the threshold.
        @param data:
        @param feature_index:
        @param threshold:
        @param option:
        @return:
        """
        # Initialize an empty list to store the filtered data
        filtered_data = []
        for data_entry in data:
            # If the option is 'S', check if the feature value is less than the threshold
            if option == 'S':
                if float(data_entry[feature_index]) < threshold:
                    # If the feature value is less than the threshold, add the entry to the filtered data
                    filtered_data.append(data_entry)
            else:
                # If the option is not 'S', check if the feature value is greater than the threshold
                if float(data_entry[feature_index]) > threshold:
                    # If the feature value is greater than the threshold, add the entry to the filtered data
                    filtered_data.append(data_entry)

        return filtered_data

    def calculate_entropy(self, data):
        """
        @description: calculate entropy for
        @param data:
        @return:
        """
        if self.method == 'ID3':
            # discrete values, count the bin is enough
            class_counts = np.bincount(data)
            probabilities = class_counts / len(data)
            entropy = -np.sum([p * np.log2(p) for p in probabilities if p > 0])
            return entropy
        elif self.method == 'C4.5':
            # Calculate the total number of entries in the data
            total_entries = len(data)
            # Initialize a dictionary to store the frequency of each label
            label_frequencies = {}
            # Iterate over each entry in the data
            for data_entry in data:
                # Get the label of the entry
                data_label = data_entry[-1]
                # If the label is not in the dictionary, initialize it with 0
                if data_label not in label_frequencies:
                    label_frequencies[data_label] = 0
                # Increment the count of the label in the dictionary
                label_frequencies[data_label] += 1
            # Initialize entropy with 0
            calculated_entropy = 0
            # Calculate the entropy
            for label_count in label_frequencies.values():
                # Calculate the probability of the label
                probability = label_count / total_entries
                # Update the entropy
                calculated_entropy -= probability * math.log2(probability)
            # Return the calculated entropy
            return calculated_entropy

        else:
            raise Exception("No such method!")

    def calculate_information_gain(self, X, y, feature):
        """
        @description: calculate the information and store in a dictionary for future looking up
        @param X:
        @param y:
        @param feature:
        @return:
        """
        total_entropy = self.calculate_entropy(y)
        values, counts = np.unique(X[feature], return_counts=True)
        weighted_entropy = np.sum(
            [(counts[i] / np.sum(counts)) * self.calculate_entropy(y[X[feature] == v]) for i, v in enumerate(values)])
        # Calculate the difference to measure the information gain
        information_gain = total_entropy - weighted_entropy
        return information_gain

    def best_feature_to_split_id3(self, X, y):
        """
        @description :Find the best feature to split which has the biggest information gain
        @param X:
        @param y:
        @return: the name of the feature has the biggest information gain
        """
        information_gains = {
            feature: self.calculate_information_gain(X, y, feature)
            for feature in X.columns}
        # max on biggest value of information gain(not the key) in the dictionary
        return max(information_gains, key=information_gains.get)

    def best_feature_to_split_c4_5(self, X, fes_prpers):
        """
        This function chooses the best feature for splitting the dataset based on information gain.
        It correctly handles both discrete and continuous features.
        """
        num_of_features = len(fes_prpers)
        base_entropy_value = self.calculate_entropy(X)
        max_info_gain = 0.0
        best_feature_index = -1
        best_partition_val = None
        for i in range(num_of_features):
            feature_values = [example[i] for example in X]
            unique_feature_values = set(feature_values)
            entropy_value = 0.0
            best_partition_val_i = None
            if fes_prpers[i] == 0:  # Discrete feature
                for value in unique_feature_values:
                    filtered_data = self.filter_data(X, value, i)
                    probability = len(filtered_data) / float(len(X))
                    entropy_value += probability * self.calculate_entropy(filtered_data)
            else:  # Continuous feature
                sorted_unique_feature_values = list(unique_feature_values)
                sorted_unique_feature_values.sort()
                min_entropy_value = 10000
                for j in range(len(sorted_unique_feature_values) - 1):
                    partition_val = (float(sorted_unique_feature_values[j]) + float(
                        sorted_unique_feature_values[j + 1])) / 2
                    left_data = self.filter_data_by_threshold(X, i, partition_val, 'S')
                    right_data = self.filter_data_by_threshold(X, i, partition_val, 'T')
                    left_probability = len(left_data) / float(len(X))
                    right_probability = len(right_data) / float(len(X))
                    entropy_val = left_probability * self.calculate_entropy(
                        left_data) + right_probability * self.calculate_entropy(
                        right_data)
                    if entropy_val < min_entropy_value:
                        min_entropy_value = entropy_val
                        best_partition_val_i = partition_val
                entropy_value = min_entropy_value
            information_gain = base_entropy_value - entropy_value
            if information_gain > max_info_gain:
                max_info_gain = information_gain
                best_feature_index = i
                best_partition_val = best_partition_val_i
        return best_feature_index, best_partition_val

    def id3(self, X, y, original_data, features, parent_node_class=None):
        """
        @description :ID3 algorithm
        @param X:
        @param y:
        @param original_data:
        @param features:
        @param parent_node_class:
        @return:
        """
        # Firstly, it can stop at three conditions(Ignore the situation that the tree reaches a certain depth):
        if len(np.unique(y)) <= 1:
            # 1. If there is only one class left, return it
            return np.unique(y)[0]
        elif len(y) == 0:
            # 2. If there is no data left, return the most common class
            return np.unique(original_data["好瓜"])[
                np.argmax(np.unique(original_data["好瓜"],
                                    return_counts=True)[1])]
        elif len(features) == 0:
            # 3. If there is no feature left, return the parent node class（which is the most common class）
            return parent_node_class
        else:
            # Then, If none of the above conditions is satisfied, continue to split
            parent_node_class = np.unique(y)[
                np.argmax(np.unique(y, return_counts=True)[1])]
            best_feature = self.best_feature_to_split_id3(X, y)
            tree = {best_feature: {}}
            features = [i for i in features if i != best_feature]
            for value in np.unique(X[best_feature]):
                sub_data = X.where(X[best_feature] == value).dropna()
                subtree = self.id3(sub_data, y[sub_data.index], original_data, features, parent_node_class)
                tree[best_feature][value] = subtree
            return tree

    def c4_5(self, X, fes, fe_propers):
        """
        @description: C4.5 algorithm
        @param X:
        @param fes:
        @param fe_propers:
        @return:
        """
        # Get the class labels for each data entry
        classList = [example[-1] for example in X]
        # If all class labels are the same, return the class label
        if classList.count(classList[0]) == len(classList):
            return classList[0]
        # If there is only one feature left, return the most common class label
        if len(X[0]) == 1:
            return self.most_common(classList)

        # Choose the best feature and partition value for splitting
        bestFeature, bestPartitionValue = self.best_feature_to_split_c4_5(X, fe_propers)

        # If no best feature is found, return the most common class label
        if bestFeature == -1:
            return self.most_common(classList)

        # If the best feature is discrete
        if fe_propers[bestFeature] == 0:
            bestFeatureLabel = fes[bestFeature]
            Tree = {bestFeatureLabel: {}}
            newLabels = copy.copy(fes)
            newLabelProperties = copy.copy(fe_propers)
            del (newLabels[bestFeature])
            del (newLabelProperties[bestFeature])
            featureValues = [example[bestFeature] for example in X]
            uniqueValues = set(featureValues)
            for value in uniqueValues:
                subLabels = newLabels[:]
                subLabelProperties = newLabelProperties[:]
                Tree[bestFeatureLabel][value] = self.c4_5(self.filter_data(X, value, bestFeature), subLabels,
                                                          subLabelProperties)
        else:  # If the best feature is continuous
            bestFeatureLabel = fes[bestFeature] + '<' + str(bestPartitionValue)
            Tree = {bestFeatureLabel: {}}
            subLabels = fes[:]
            subLabelProperties = fe_propers[:]
            # Build the left subtree
            leftValue = '是'
            Tree[bestFeatureLabel][leftValue] = self.c4_5(
                self.filter_data_by_threshold(X, bestFeature, bestPartitionValue, 'S'), subLabels,
                subLabelProperties)
            # Build the right subtree
            rightValue = '否'
            Tree[bestFeatureLabel][rightValue] = self.c4_5(
                self.filter_data_by_threshold(X, bestFeature, bestPartitionValue, 'T'), subLabels,
                subLabelProperties)
        return Tree

    def post_prune(self, node, validation_data):
        """
        @description: Post-pruning method for the decision tree
        @param node: current node
        @param validation_data: validation data set
        @return: pruned decision tree
        """
        # Check if the current node is a leaf node
        if not isinstance(node, dict):
            return node

        # Recursively prune child nodes
        for key in node.keys():
            node[key] = self.post_prune(node[key], validation_data)

        # Check if the current node can be pruned
        original_accuracy = self.calculate_accuracy(validation_data['label'], self.predict(validation_data))
        pruned_node = self.most_common(validation_data['label'])
        pruned_accuracy = self.calculate_accuracy(validation_data['label'],
                                                  self.predict(validation_data, pruned_node))

        # If the accuracy improves after pruning, prune the node
        if pruned_accuracy >= original_accuracy:
            return pruned_node
        else:
            return node


    def fit(self, X, y):
        """
        @description: train the model based on different given methods
        @param X:
        @param y:
        @return:
        """

        if self.method == 'ID3':
            features = X.columns.tolist()
            self.tree = self.id3(X, y, pd.concat([X, y], axis=1), features)
            if self.puring:
                self.tree = self.post_prune(self.tree, pd.concat([pd.DataFrame(X[:, :-1], columns=self.features), y], axis=1))
        elif self.method == 'C4.5':
            self.tree = self.c4_5(X, self.features, self.features_properties)
            if self.puring:
                self.tree = self.post_prune(self.tree, pd.concat([pd.DataFrame(X[:, :-1], columns=self.features), y], axis=1))
        else:
            raise Exception("No such method!")

    def predict(self, X, tree =None):
        """
        @description: using the re-trained model to predict the result
        @param X:
        @return:
        """
        if self.method == 'ID3':
            # apply predict_single on every row of X
            predictions = X.apply(self.predict_single, axis=1, args=(tree,))
            return predictions
        elif self.method == 'C4.5':
            predictions = []
            for x in X:
                predictions.append(self.predict_single(tree, self.features, self.features_properties, x))
        else:
            raise Exception("No such method!")

        return predictions

    def predict_single(self, DecisionTree, fes, fe_propers, test_data):
        """
        @description: using the re-trained model to predict the result
        @param DecisionTree: the whole decision tree or subtree
        @param fes: feature labels
        @param fe_propers: feature properties
        @param test_data: the test data
        @return:
        """
        if self.method == 'ID3':
            # Traverses the decision tree on the values in 'x' until it reaches a leaf node.
            tree = DecisionTree
            while isinstance(tree, dict):
                root = next(iter(tree))
                tree = tree[root][test_data[root]]
            # The class label of the leaf node is returned as the prediction for 'x'.
            return tree
        elif self.method == 'C4.5':
            # Get the first key of the input tree
            firstKey = list(DecisionTree.keys())[0]
            featureName = firstKey
            # Check if the feature is continuous
            splitIndex = str(firstKey).find('<')
            if splitIndex > -1:
                featureName = str(firstKey)[:splitIndex]
            # Get the subtree corresponding to the first key
            subTree = DecisionTree[firstKey]
            # Get the index of the feature in the feature labels list
            featureIndex = fes.index(featureName)
            predictedLabel = None
            for key in subTree.keys():
                if fe_propers[featureIndex] == 0:
                    if test_data[featureIndex] == key:
                        if type(subTree[key]).__name__ == 'dict':
                            predictedLabel = self.predict_single(subTree[key], fes, fe_propers, test_data)
                        else:
                            predictedLabel = subTree[key]
                else:
                    partitionValue = float(str(firstKey)[splitIndex + 1:])
                    if test_data[featureIndex] < partitionValue:
                        if type(subTree['是']).__name__ == 'dict':
                            predictedLabel = self.predict_single(subTree['是'], fes, fe_propers, test_data)
                        else:
                            predictedLabel = subTree['是']
                    else:
                        if type(subTree['否']).__name__ == 'dict':
                            predictedLabel = self.predict_single(subTree['否'], fes, fe_propers, test_data)
                        else:
                            predictedLabel = subTree['否']

            return predictedLabel
        else:
            raise Exception("No such method!")

    def calculate_accuracy(self, y_true, y_pred):
        """
        @description: calculate accuracy for true labels and predicted ones
        @param y_true:
        @param y_pred:
        @return:
        """
        return np.sum(y_true == y_pred) / len(y_true)

然后我**重新利用剪枝的方法构造对应的ID3和C4.5决策树**，并进行预测，结果如下所示：

In [11]:
print("------------------ID3 Decision Tree with Puring------------------")
# Create Decision Tree Model and train it
Decision_Tree_Classifier_ID3_pured = DecisionTree("ID3", features, feature_properties, puring=True)
Decision_Tree_Classifier_ID3_pured.fit(X_train, y_train)
# Predict and output the accuracy and error rate
y_pred = Decision_Tree_Classifier_ID3_pured.predict(X_test)
accuracy = Decision_Tree_Classifier_ID3_pured.calculate_accuracy(y_test, y_pred)
error_rate = (1 - accuracy) * 100
accuracy = accuracy * 100
print("Classify accuracy for Decision Tree Classifier ID3 with Puring is: {:.2f}%".format(accuracy))
print("Classify error rate for Decision Tree Classifier ID3 with Puring is: {:.2f}%".format(error_rate))



print("------------------C4.5 Decision Tree with Puring------------------")
# Create Decision Tree Model and train it
Decision_Tree_Classifier_C4_5_pured = DecisionTree("C4.5", features, feature_properties, puring=True)
Decision_Tree_Classifier_C4_5_pured.fit(train2, df_train2['好瓜'])
# Predict and output the accuracy
y_pred = Decision_Tree_Classifier_C4_5_pured.predict(test2)
accuracy = Decision_Tree_Classifier_C4_5_pured.calculate_accuracy(df_test2['好瓜'], y_pred)
error_rate = (1 - accuracy) * 100
accuracy = accuracy * 100
print("Classify accuracy for Decision Tree Classifier C4.5 with Puring is: {:.2f}%".format(accuracy))
print("Classify error rate for Decision Tree Classifier C4.5 with Puring is: {:.2f}%".format(error_rate))

------------------ID3 Decision Tree with Puring------------------
Classify accuracy for Decision Tree Classifier ID3 with Puring is: 75.00%
Classify error rate for Decision Tree Classifier ID3 with Puring is: 25.00%
------------------C4.5 Decision Tree with Puring------------------
Classify accuracy for Decision Tree Classifier C4.5 with Puring is: 81.50%
Classify error rate for Decision Tree Classifier C4.5 with Puring is: 18.50%


### 5.3 结果分析
最后结合前面初级，中级，高级的三阶段要求，让我们对比一下不同方法的决策树已经剪枝前后的分类精度和错误率。**结果如下：**
1. **ID3决策树：**
    - 初级要求（剪枝前）：分类精度为70%，错误率为30%
    - 高级要求（剪枝后）：分类精度为75%，错误率为25%
2. **C4.5决策树：**
    - 中级要求（剪枝前）：分类精度为80%，错误率为20%
    - 高级要求（剪枝后）：分类精度为81.5%，错误率为18.5%
3. **ID3决策树分析**
    - **剪枝前**：ID3决策树在小数据集上容易过拟合，因为它倾向于创建深且复杂的树，尤其是在有许多属性的情况下。这可能导致分类精度较低，因为树可能会对训练数据中的噪声和异常值过度敏感。
    - **剪枝后**：应用后剪枝后，精度提高到了75%。这表明剪枝帮助去除了一些过拟合的部分，使树在未知数据上表现得更好。在小数据集上，剪枝可以帮助模型更好地泛化，避免对训练数据的特殊特征过度适应。
4. **C4.5 决策树分析**
    - **剪枝前**：C4.5决策树本身就是ID3的改进版本，它通过使用信息增益比来减少对具有多个值的属性的偏见，并能处理连续属性。这可能是为什么C4.5在剪枝前就表现出更高的分类精度（80%）。
    - **剪枝后**：经过剪枝后C4.5的精度略微提高到了81.5%。这个小幅的提升可能是因为数据集本身不大，剪枝移除了一些不必要的分支，从而提高了模型的泛化能力。
5. **总体分析：**
在数据集较小时，决策树倾向于过拟合，因为它们可能会对训练数据中的每个细节都进行学习，包括噪声和异常值。后剪枝通过减少树的复杂性来提高模型的泛化能力。**在像是本次实验这种样本量较少的情况下，剪枝可能带来的结果是不稳定的。** 剪枝对于改善两种决策树的性能都是有效的，尽管对C4.5决策树的影响相对较小。这可能是因为C4.5自身就包含了一些减少过拟合的机制（如信息增益比），使得它在未剪枝的情况下也能相对较好地处理小数据集。


## 实验心得和收获

* 本次实验中，我首先对给定的数据集进行预分析和处理，**发现了数据集量较小的情况，并对后面的可能进行了预测，果不其然，后面的实验中也出现了剪枝不稳定的情况。**

* 对上课老师所讲到的决策树相关算法有了更深刻的理解。**亲自编写了ID3和C4.5决策树对我锻炼极大。**

* 通过将ID3和C4.5决策树成功地**将二者封装成一个类，提供了很方便的训练和预测的接口。对我的编程能力也有了很大的提升。**

* 最后在剪枝的过程中，我也**对决策树的过拟合问题有了更深刻的理解。**

* 最后我还通过进一步探索和挖掘，分析使用剪枝前后ID3和C4.5决策树性能的差异。**更加加深了我对算法的理解。**

总的来说我收获颇丰，也是最后一次实现的个人的机器学习作业。**在这里想要对传授我们机器学习这么底层和原理知识的卫老师表达感激，对一直帮我答疑的助教学长闫乘玮和学姐陈曦冉同样表达十分的感谢！这一学期的编程作业我收获极大，后面的大作业我也会和组员们努力完成哒~**